In [1]:
import sagemaker
import boto3
import os

sagemaker_session = sagemaker.Session()
print('SageMaker version: ' + sagemaker.__version__)

# bucket = sagemaker_session.default_bucket()
bucket = 'prod-tpgt-knowledge-lake-sandpit-v1'
prefix = 'zihan-research/dialogue-state-tracking/PPTOD-AL'
# base_job_name = 'KAGE-DS-L4P4K2-RandomTurn'
# base_job_name = 'KAGE-DS-L4P4K2-LastTurn'

# role = sagemaker.get_execution_role()
# checkpoint_s3_path = f's3://{bucket}/{prefix}/{base_job_name}/checkpoints'

# print('Checkpointing Path: {}'.format(checkpoint_s3_path))

SageMaker version: 2.86.2


In [ ]:
# # upload files to s3
# def upload_to_s3(file_name, bucket, object_name):
#     if object_name is None:
#         object_name = os.path.basename(file_name)
    
#     s3_client = boto3.client('s3')
#     print(object_name)
#     s3_client.upload_file(file_name, bucket, object_name)

In [ ]:
# upload data to S3

#### Selected Turns

In [36]:
!ls selected_turns

epoch_5_round_3_dev_joint_accuracy_36.24  selected_turn_id.csv
model.tar.gz				  test_acc_per_round.csv


In [37]:
# clear selected_turns folder
!rm -r selected_turns
!ls

checkpoints		       DST	overview.png
CODE_OF_CONDUCT.md	       E2E_TOD	Pretraining
CONTRIBUTING.md		       IC	README.md
data			       LICENSE	requirements.txt
download_trained_output.ipynb  NOTICE	sagemaker_train.ipynb


In [38]:
key = 'zihan-research/dialogue-state-tracking/PPTOD-AL/Select-Turn/PPTOD-base-MaxEntropy/mwz21/model/PPTOD-MWZ21-ME-B24D2000Bud0-S164-2022-06-17-12-54-34-041/output'

In [39]:
%%time
# download the output.tar.gz to selected_turns folder
sagemaker_session.download_data(path='selected_turns', bucket=bucket, key_prefix=key)

CPU times: user 2.39 s, sys: 2.21 s, total: 4.6 s
Wall time: 4.88 s


In [40]:
%%time
# Extract the files from output.tar.gz to checkpoints directory
!tar -zxvf selected_turns/model.tar.gz -C selected_turns

selected_turn_id.csv
test_acc_per_round.csv
epoch_11_round_3_dev_joint_accuracy_36.04/
epoch_11_round_3_dev_joint_accuracy_36.04/test_joint_accuracy_49.21.json
epoch_11_round_3_dev_joint_accuracy_36.04/added_tokens.json
epoch_11_round_3_dev_joint_accuracy_36.04/spiece.model
epoch_11_round_3_dev_joint_accuracy_36.04/dev_joint_accuracy_36.04.json
epoch_11_round_3_dev_joint_accuracy_36.04/tokenizer_config.json
epoch_11_round_3_dev_joint_accuracy_36.04/special_tokens_map.json
epoch_11_round_3_dev_joint_accuracy_36.04/pytorch_model.bin
epoch_11_round_3_dev_joint_accuracy_36.04/config.json
CPU times: user 49.9 ms, sys: 26.6 ms, total: 76.5 ms
Wall time: 7.85 s


In [41]:
# upload files to s3
def upload_to_s3(file_name, bucket, object_name):
    if object_name is None:
        object_name = os.path.basename(file_name)
    
    s3_client = boto3.client('s3')
    print(object_name)
    s3_client.upload_file(file_name, bucket, object_name)

def get_filenames(directory_path):
    filenames = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".json") or filename.endswith(".csv"): 
            filenames.append(filename)
    
    return filenames

In [42]:
selected_turn_filenames = get_filenames("./selected_turns")
selected_turn_paths = [f'selected_turns/{path}' for path in selected_turn_filenames]
selected_turn_paths

['selected_turns/test_acc_per_round.csv',
 'selected_turns/selected_turn_id.csv']

In [43]:
key = 'zihan-research/dialogue-state-tracking/PPTOD-AL/Select-Turn/PPTOD-base-MaxEntropy/mwz21/model/PPTOD-MWZ21-ME-B24D2000Bud0-S164-2022-06-17-12-54-34-041/output'

for i in list(range(len(selected_turn_paths))): 
    upload_to_s3(selected_turn_paths[i], bucket, f'{key}/{selected_turn_filenames[i]}')

zihan-research/dialogue-state-tracking/PPTOD-AL/Select-Turn/PPTOD-base-MaxEntropy/mwz21/model/PPTOD-MWZ21-ME-B24D2000Bud0-S164-2022-06-17-12-54-34-041/output/test_acc_per_round.csv
zihan-research/dialogue-state-tracking/PPTOD-AL/Select-Turn/PPTOD-base-MaxEntropy/mwz21/model/PPTOD-MWZ21-ME-B24D2000Bud0-S164-2022-06-17-12-54-34-041/output/selected_turn_id.csv


In [86]:
# move the selected turn csv to Src/selected_turn
!cp 'selected_turns/selected_turn_id.csv' 'Src/selected_turns/selected_turn_id.csv'

In [87]:
!cp -r 'selected_turns/tokenizer/' 'Src/selected_turns/tokenizer/'

In [88]:
!cp -r 'selected_turns/model_best.pth.tar' 'Src/selected_turns/model_best.pth.tar'

Local

In [28]:
# !cp 'selected_turns/selected_turn_id.csv' 'Experiments/mwz20_debug5/train/saved_model/selected_turn_id.csv'

In [29]:
# !cp -r 'selected_turns/tokenizer/' 'Experiments/mwz20_debug5/train/saved_model/tokenizer/'

In [30]:
# !cp -r 'selected_turns/model_best.pth.tar' 'Experiments/mwz20_debug5/train/saved_model/model_best.pth.tar'

#### Run Testing

In [11]:
!ls DST/results

epoch_6_dev_joint_accuracy_7.07  model.tar.gz  test.csv


In [12]:
# clear selected_turns folder
!rm -r DST/results
!ls

checkpoints		       DST	overview.png
CODE_OF_CONDUCT.md	       E2E_TOD	Pretraining
CONTRIBUTING.md		       IC	README.md
data			       LICENSE	requirements.txt
download_trained_output.ipynb  NOTICE	sagemaker_train.ipynb


In [13]:
key = 'zihan-research/dialogue-state-tracking/PPTOD-AL/Train/PPTOD-base-LastTurn/mwz20/model/Train-PPTOD-base-LT-S625-2022-06-11-16-18-17-014/output/model.tar.gz'

In [14]:
%%time
# download the output.tar.gz to selected_turns folder
sagemaker_session.download_data(path='DST/results', bucket=bucket, key_prefix=key)

CPU times: user 2.94 s, sys: 2.67 s, total: 5.61 s
Wall time: 2.98 s


In [15]:
%%time
# Extract the files from output.tar.gz to checkpoints directory
!tar -zxvf DST/results/model.tar.gz -C DST/results

epoch_6_dev_joint_accuracy_8.08/
epoch_6_dev_joint_accuracy_8.08/config.json
epoch_6_dev_joint_accuracy_8.08/added_tokens.json
epoch_6_dev_joint_accuracy_8.08/test_joint_accuracy_14.14.json
epoch_6_dev_joint_accuracy_8.08/pytorch_model.bin
epoch_6_dev_joint_accuracy_8.08/spiece.model
epoch_6_dev_joint_accuracy_8.08/tokenizer_config.json
epoch_6_dev_joint_accuracy_8.08/special_tokens_map.json
epoch_6_dev_joint_accuracy_8.08/dev_joint_accuracy_8.08.json
CPU times: user 93.6 ms, sys: 19.8 ms, total: 113 ms
Wall time: 7.76 s


In [95]:
!ls checkpoints

model_best.pth.tar  output.tar.gz  tokenizer


In [36]:
# !mv 'checkpoints/checkpoint-epoch-9.pth.tar' 'checkpoints/model_best.pth.tar'